In [16]:
import requests
import json
import numpy as np
import pandas as pd
import csv
import time
from datetime import datetime

In [3]:
movimientos_df = pd.read_csv('movimientos_general_con_frecuencias.csv', sep=";", header=0)
rutas_originales = pd.read_csv('estaciones_bicimad_totales_agregado.csv', sep=";", header=0)

## 5.1 Rutas por línea

In [87]:
dict_history = {'Lineas': []} 

In [88]:
for index, row in movimientos_df.iterrows():
    band = False
    for d2 in dict_history['Lineas']:
        if (d2['Linea'] == row["id_line"]):
            band2 = False
            for d3 in d2['rutas']:
                if d3['idunplug_station'] == row["idunplug_station"] and d3["idplug_station"] == row["idplug_station"]:
                    d3['num_viajes'] = d3['num_viajes'] + 1 
                    d3['tiempo_viaje'] = d3['tiempo_viaje'] + row["travel_time"]
                    band2 = True
            if not band2: d2['rutas'].append({"idunplug_station":row["idunplug_station"],"idplug_station":row["idplug_station"],"id_header_A":row["id_header_A"],"id_header_B":row["id_header_B"],"num_viajes":1,"tiempo_viaje":row["travel_time"]})
            band = True  
    if not band: dict_history['Lineas'].append({"Linea":row["id_line"],"freq_min":row["dir1_min_freq"],"freq_max":row["dir1_max_freq"],"rutas":[{"idunplug_station":row["idunplug_station"],"id_header_A":row["id_header_A"],"id_header_B":row["id_header_B"],"idplug_station":row["idplug_station"],"num_viajes":1,"tiempo_viaje":row["travel_time"]}]})

##### Líneas encontradas en los movimientos

In [90]:
for l in dict_history['Lineas']:
    print("Linea: " + str(l["Linea"]))

Linea: 3
Linea: 19
Linea: 26
Linea: 43
Linea: 46
Linea: 61
Linea: 68
Linea: 69
Linea: 156
Linea: 361
Linea: 362
Linea: 601
Linea: 215


##### Número de pares (rutas) de estaciones BiciMAD por línea

In [91]:
for l in dict_history['Lineas']:
    print("Línea: " + str(l["Linea"]) + ": " + str(len(l["rutas"])))

Línea: 3: 2
Línea: 19: 5
Línea: 26: 8
Línea: 43: 7
Línea: 46: 18
Línea: 61: 19
Línea: 68: 12
Línea: 69: 12
Línea: 156: 6
Línea: 361: 67
Línea: 362: 6
Línea: 601: 24
Línea: 215: 11


##### Análisis por rutas

In [158]:
aux = []
estaciones_bicimad_ida = []
estaciones_bicimad_vuelta = []
cont = 0
for l in dict_history['Lineas']:
    num_viajes = 0
    tiempo_viajes = 0
    lineas_repetidas = []
    for r in l["rutas"]:
        tiempo_viajes += r['tiempo_viaje']
        num_viajes += r['num_viajes']
        media = round(r['tiempo_viaje']/r['num_viajes']/60,2)
        media_autobus = round((l["freq_min"] + l["freq_max"])/2,2)
        cont = cont + 1
        if ((r['idplug_station'],r['idunplug_station']) in lineas_repetidas):
            cont = cont - 1
            estaciones_bicimad_vuelta.append([l["Linea"],r['id_header_B'],r['idplug_station'],r['id_header_A'],r['idunplug_station']])
        else:
            estaciones_bicimad_ida.append([l["Linea"],r['id_header_A'],r['idunplug_station'],r['id_header_B'],r['idplug_station']])
        lineas_repetidas.append((r['idunplug_station'],r['idplug_station']))
        aux.append([l["Linea"],r['id_header_A'],r['idunplug_station'],r['id_header_B'],r['idplug_station'],r['num_viajes'],media,media_autobus])
pd.DataFrame(aux).to_csv('agrupamiento_linea.csv',sep=";",header=['id_line','id_header_A','idunplug_station','id_header_B','idplug_station','num_viajes','media','media_autobus'],index=False)

##### Número de rutas totales encontradas

In [19]:
agrupamiento_df = pd.read_csv('agrupamiento_linea.csv', sep=";", header=0)

In [149]:
agrupamiento_df.count()[0]

197

##### Número de rutas de ida encontradas

In [137]:
estaciones_ida_df = pd.DataFrame(estaciones_bicimad_ida)
estaciones_ida_df.columns =['id_line','id_header_A','idunplug_station','id_header_B','idplug_station']

In [148]:
estaciones_ida_df.count()[0]

103

##### Número de rutas de ida vuelta

In [139]:
estaciones_vuelta_df = pd.DataFrame(estaciones_bicimad_vuelta)
estaciones_vuelta_df.columns = ['id_line','id_header_B','idplug_station','id_header_A','idunplug_station']

In [146]:
estaciones_vuelta_df.count()[0]

94

##### Estaciones originales

In [145]:
rutas_originales.count()[0]

165

### 5.1.1 Ocupación de bicicleta respecto a un autobús.

In [20]:
agrupamiento_df['bici_mejor'] = np.where(agrupamiento_df['media'] <= agrupamiento_df['media_autobus'] , 1, 0)
round((agrupamiento_df['bici_mejor'].value_counts()/agrupamiento_df['bici_mejor'].count())*100,2)

0    71.07
1    28.93
Name: bici_mejor, dtype: float64

##### Número de viajes por línea

In [243]:
num_viajes = movimientos_df.groupby('id_line').count()
num_viajes.reset_index(inplace = True)
num_viajes.rename(columns={'id_header_A': 'num_viajes'}, inplace=True)
num_viajes[['id_line','num_viajes']].sort_values('num_viajes',ascending = False)

,id_line,num_viajes
12,601,6955
4,46,4197
10,361,4102
2,26,2856
5,61,1708
11,362,1658
7,69,1603
6,68,1283
8,156,1020
3,43,621


##### Número de rutas por línea

In [244]:
num_rutas = agrupamiento_df.groupby('id_line').count()
num_rutas.reset_index(inplace = True)
num_rutas.rename(columns={'id_header_A': 'num_rutas'}, inplace=True)
num_rutas[['id_line','num_rutas']].sort_values('num_rutas',ascending = False)

,id_line,num_rutas
10,361,67
12,601,24
5,61,19
4,46,18
6,68,12
7,69,12
9,215,11
2,26,8
3,43,7
8,156,6


##### Número de rutas donde es mejor la bicicleta como medio de transporte

In [ ]:
infoline_df = pd.read_csv('infoline.csv', sep=";", header=0)

In [23]:
agp_df = agrupamiento_df[agrupamiento_df['bici_mejor'] == 1].groupby('id_line').count().sort_values('id_header_A',ascending=False)
agp_df.reset_index(inplace = True)
agp_df.rename(columns={'id_header_A': 'conteo','id_line': 'line'}, inplace=True)
agp_df[['line','conteo']].merge(infoline_df[['line','longLine1','longLine2']], on='line',how='left')

,line,conteo,longLine1,longLine2
0,601,20,2298,2420
1,215,11,2471,2890
2,46,10,6304,6913
3,26,5,5651,6330
4,361,5,4827,4811
5,362,5,5276,5415
6,43,1,7787,7630


### 5.1.2 Tipos de usuario en general

##### Porcentaje

In [4]:
usuarios_df = movimientos_df.groupby('user_type').count()
(usuarios_df['user_day_code']/usuarios_df['user_day_code'].sum())*100

user_type
1    98.720874
2     1.279126
Name: user_day_code, dtype: float64

##### Número de casos

In [8]:
movimientos_df.groupby('user_type').count()['user_day_code']

user_type
1    26395
2      342
Name: user_day_code, dtype: int64

###### Tipos de usuario en función de las rutas por línea

In [51]:
mov_user_type = movimientos_df.groupby(['id_line','user_type']).count()
sum_mov_user_type = mov_user_type.groupby(['id_line', 'user_type']).agg({'user_day_code': 'sum'})
sum_mov_user_type = sum_mov_user_type.groupby(level=0).apply(lambda x: round(100 * x / float(x.sum()),2))
sum_mov_user_type.reset_index(inplace = True)
sum_mov_user_type['user_type'] = np.where(sum_mov_user_type['user_type'] != 1 , "Anual", "Ocasional")
sum_mov_user_type.rename(columns={'user_day_code': 'porcentaje'}, inplace=True)
sum_mov_user_type = sum_mov_user_type.pivot(index='id_line', columns='user_type', values='porcentaje')
sum_mov_user_type

user_type,Anual,Ocasional
id_line,,
3,NaN,100.00
19,2.15,97.85
26,0.91,99.09
43,1.13,98.87
46,1.45,98.55
61,0.53,99.47
68,0.70,99.30
69,0.62,99.38
156,1.37,98.63


### 5.1.3 Rango de edades de los usuarios

In [197]:
edades_df = movimientos_df.groupby('ageRange').count()
edades_df['rango_texto'] = ['Sin edad','0 y 16 años','17 y 18 años','19 y 26 años','27 y 40 años','41 y 65 años','66 años o más']
edades_df ['porcentaje'] = round((edades_df['user_day_code']/edades_df['user_day_code'].sum())*100,2)
edades_df.reset_index(inplace=True)
edades_df['id'] = edades_df.index
edades_df[['rango_texto','porcentaje']].sort_values('porcentaje',ascending = False)[['rango_texto','porcentaje']]

,rango_texto,porcentaje
4,27 y 40 años,35.00
0,Sin edad,34.46
5,41 y 65 años,18.87
3,19 y 26 años,10.29
2,17 y 18 años,0.98
6,66 años o más,0.24
1,0 y 16 años,0.15


###### Rango de edades de los usuarios en función de las rutas por línea

In [52]:
mov_age_range = movimientos_df.groupby(['id_line','ageRange']).count()
sum_mov_age_range = mov_age_range.groupby(['id_line', 'ageRange']).agg({'user_day_code': 'sum'})
age_res = sum_mov_age_range.groupby(level=0).apply(lambda x: round(100 * x / float(x.sum()),2))
age_res.reset_index(inplace = True)
edades_df.reset_index(inplace=True)
age_res = age_res.merge(edades_df[['ageRange','rango_texto']], on='ageRange',how='left')
pd.set_option('display.max_rows', None)
age_res.rename(columns={'user_day_code': 'porcentaje'}, inplace=True)
age_res.sort_values(['id_line','porcentaje'],ascending=False)
age_res = age_res[['id_line','porcentaje','rango_texto']]
age_res = age_res.pivot(index='id_line', columns='rango_texto', values='porcentaje')
age_res

rango_texto,0 y 16 años,17 y 18 años,19 y 26 años,27 y 40 años,41 y 65 años,66 años o más,Sin edad
id_line,,,,,,,
3,NaN,NaN,0.36,85.87,7.97,NaN,5.80
19,NaN,5.91,1.88,41.13,11.83,NaN,39.25
26,0.04,0.42,6.37,42.58,9.49,0.18,40.93
43,0.81,0.48,30.92,15.94,25.93,NaN,25.93
46,0.10,1.24,9.79,27.14,22.42,0.43,38.88
61,0.18,1.11,17.51,12.47,33.08,0.47,35.19
68,NaN,0.16,11.22,44.89,19.41,0.23,24.08
69,NaN,0.19,9.54,47.16,13.91,0.19,29.01
156,0.49,1.37,6.86,39.61,15.98,0.29,35.39


## 5.2 Rutas por línea en función del tipo de día

In [323]:
dict_history_dt = {'Lineas': []} 

In [324]:
for index, row in movimientos_df.iterrows():
    band = False
    for d2 in dict_history_dt['Lineas']:
        if (d2['Linea'] == row["id_line"]):
            band2 = False
            for d3 in d2['day_types']:
                if d3['day_type'] == row["day_type"]:
                    d3['num_viajes'] = d3['num_viajes'] + 1 
                    d3['tiempo_viaje'] = d3['tiempo_viaje'] + row["travel_time"]
                    band2 = True
            if not band2:
                d2['day_types'].append({"day_type":row["day_type"],"id_header_A":row["id_header_A"],"id_header_B":row["id_header_B"],"num_viajes":1,"tiempo_viaje":row["travel_time"]})
            band = True  
    if not band:
        dict_history_dt['Lineas'].append({"Linea":row["id_line"],"day_types":[{"day_type":row["day_type"],"id_header_A":row["id_header_A"],"id_header_B":row["id_header_B"],"num_viajes":1,"tiempo_viaje":row["travel_time"]}]})

#### Análisis de rutas

In [334]:
aux = []
for l in dict_history_dt['Lineas']:
    num_viajes = 0
    tiempo_viajes = 0
    for r in l["day_types"]:
        num_viajes += r['tiempo_viaje']
        tiempo_viajes += r['num_viajes']
        aux.append([l["Linea"],r['day_type'] ,r['num_viajes'],round(r['tiempo_viaje']/r['num_viajes']/60,2)])
pd.DataFrame(aux).to_csv('agrupamiento_linea_tipo_dia.csv',sep=";",header=['id_line','day_type','num_viajes','media'],index=False)

In [151]:
agrupamiento_tipo_dia_df = pd.read_csv('agrupamiento_linea_tipo_dia.csv', sep=";", header=0)

##### Número de viajes por tipo de día

In [77]:
tipo_dia_df = agrupamiento_tipo_dia_df.groupby('day_type').sum()
round((tipo_dia_df['num_viajes']/tipo_dia_df['num_viajes'].sum())*100,2)

day_type
FE    14.23
LA    73.79
SA    11.98
Name: num_viajes, dtype: float64

### 5.2.1. Viajes en función de la línea

In [180]:
tipo_dia_linea = movimientos_df.groupby(['id_line','day_type']).count()
sum_mov_tipo_dia = tipo_dia_linea.groupby(['id_line', 'day_type']).agg({'user_day_code': 'sum'})
tipo_dia_linea = sum_mov_tipo_dia.groupby(level=0).apply(lambda x: round(100 * x / float(x.sum()),2))
tipo_dia_linea.rename(columns={'user_day_code': 'porcentaje'}, inplace=True)
tipo_dia_linea.reset_index(inplace = True)
tipo_dia_linea.pivot_table(values='porcentaje', index='id_line', columns='day_type', aggfunc='first')


day_type,FE,LA,SA
id_line,,,
3,3.26,94.93,1.81
19,13.17,66.13,20.70
26,16.81,71.60,11.59
43,17.87,62.96,19.16
46,12.91,76.48,10.60
61,7.14,85.60,7.26
68,17.61,67.97,14.42
69,16.97,68.25,14.78
156,19.61,66.86,13.53


### 5.2.2. Tipos de usuario por tipo de día

In [207]:
tipo_usuario_tipo_dia = movimientos_df.groupby(['day_type','user_type']).count()
sum_mov_user_type_tipo_dia = tipo_usuario_tipo_dia.groupby(['day_type', 'user_type']).agg({'user_day_code': 'sum'})
sum_mov_user_type_tipo_dia= sum_mov_user_type_tipo_dia.groupby(level=0).apply(lambda x: round(100 * x / float(x.sum()),2))
sum_mov_user_type_tipo_dia.reset_index(inplace = True)
sum_mov_user_type_tipo_dia['user_type'] = np.where(sum_mov_user_type_tipo_dia['user_type'] != 1 , "Anual", "Ocasional")
sum_mov_user_type_tipo_dia.rename(columns={'user_day_code': 'porcentaje'}, inplace=True)
sum_mov_user_type_tipo_dia = sum_mov_user_type_tipo_dia.pivot(index='day_type', columns='user_type', values='porcentaje')
sum_mov_user_type_tipo_dia

user_type,Anual,Ocasional
day_type,,
FE,2.16,97.84
LA,0.92,99.08
SA,2.47,97.53


### 5.2.3. Edad de los usuarios por tipo de día

In [205]:
tipo_usuario_tipo_dia = movimientos_df.groupby(['day_type','ageRange']).count()
sum_mov_user_type_tipo_dia = tipo_usuario_tipo_dia.groupby(['day_type', 'ageRange']).agg({'user_day_code': 'sum'})
age_tipo_res = sum_mov_user_type_tipo_dia.groupby(level=0).apply(lambda x: round(100 * x / float(x.sum()),2))
age_tipo_res.reset_index(inplace=True)
age_tipo_res.rename(columns={'user_day_code': 'porcentaje','ageRange': 'id'}, inplace=True)
age_tipo_res.sort_values(['day_type','porcentaje'],ascending=False)
age_tipo_res = pd.merge(age_tipo_res, edades_df[['id','rango_texto']], on='id')
age_tipo_res = age_tipo_res.pivot(index='day_type', columns='rango_texto', values='porcentaje')
age_tipo_res

rango_texto,0 y 16 años,17 y 18 años,19 y 26 años,27 y 40 años,41 y 65 años,66 años o más,Sin edad
day_type,,,,,,,
FE,0.18,1.34,11.93,33.40,13.80,0.34,39.00
LA,0.14,0.79,9.71,35.55,20.37,0.23,33.21
SA,0.19,1.78,11.95,33.55,15.61,0.19,36.74
